**Olist** is Brazilian e-commerce business with huge customer base.There are 9 different datasets in csv format. Pushing this data into the database will introduce new challenges, at the same time improve confidence on the data loading skill.

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import configparser
import psycopg2

In [2]:
config = configparser.ConfigParser()
config.read('clusteroli.config')

['clusteroli.config']

In [3]:
db = config['POSTGRES']['PG_DB']
user = config['POSTGRES']['PG_UNAME']
passwd = config['POSTGRES']['PG_PASS']
port = config['POSTGRES']['PG_PORT']
host = config['POSTGRES']['PG_HOST']

In [4]:
db

'olistdb'

In [5]:
credentials = "postgresql://{}:{}@{}:{}/{}".format(user,passwd,host,port,db)

#using psycopg2 to test connection since there are no tables
import psycopg2
try:
    conn = psycopg2.connect(host=host,dbname=db,user=user,password=passwd,port=port)
except Exception as e:
    print(e)
    
conn.set_session(autocommit=True)

try:
    cur = conn.cursor()
    
except:
    print(e)

In [6]:
#Helper functions to work with the database
def schemaGen(dataframe, schemaName):
    localSchema = pd.io.sql.get_schema(dataframe,schemaName)
    localSchema = localSchema.replace('TEXT','VARCHAR(255)').replace('INTEGER','NUMERIC').replace('\n','').replace('"',"")
    return "".join(localSchema)

#Using pandas read_sql for getting schema
def getSchema(tableName, credentials):
    schema = pd.read_sql("""SELECT * FROM information_schema.columns where table_name='{}'""".format(tableName),con=credentials)
    return schema

#Issue is in using pd.read_sql to write data to the database. so using psycopg2
def queryTable(query):
    try:
        schema = cur.execute(query)
        return 
    except Exception as e:
        print(e)
        
#This doesn't return anything

#Using the pd.read_sql for getting data from db
def queryBase(query):
    requiredTable = pd.read_sql(query,con=credentials)
    return requiredTable

#This returns the dataframe

In [7]:
customers = pd.read_csv("olist_customers_dataset.csv")
customers.head(2)
#Contains customer and their location
#PK is customer_unique_id << Fixed

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP


In [9]:
schemaGen(customers,'customerlocation')

'CREATE TABLE customerlocation (customer_id VARCHAR(255),  customer_unique_id VARCHAR(255),  customer_zip_code_prefix NUMERIC,  customer_city VARCHAR(255),  customer_state VARCHAR(255))'

In [11]:
createCust = """CREATE TABLE dim_location (customer_id VARCHAR(255),  customer_unique_id VARCHAR(255),  customer_zip_code_prefix NUMERIC,  customer_city VARCHAR(255),  customer_state VARCHAR(255))"""
queryTable(createCust)

In [15]:
insertCust = """COPY dim_location FROM '/var/lib/postgresql/data/olist_customers_dataset.csv'
            DELIMITER ',' CSV HEADER"""
queryTable(insertCust)

In [16]:
queryBase("""SELECT * FROM dim_location LIMIT 5""")

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409.0,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790.0,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151.0,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775.0,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056.0,campinas,SP


In [17]:
orders = pd.read_csv("olist_order_items_dataset.csv")
orders.head(5)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [19]:
schemaGen(orders,'dim_orders')
#There are multiple ids, but the order_id will be the unique key

'CREATE TABLE dim_orders (order_id VARCHAR(255),  order_item_id NUMERIC,  product_id VARCHAR(255),  seller_id VARCHAR(255),  shipping_limit_date VARCHAR(255),  price REAL,  freight_value REAL)'

In [20]:
createOrd = """CREATE TABLE dim_orders (order_id VARCHAR(255),  order_item_id NUMERIC,  product_id VARCHAR(255),  seller_id VARCHAR(255),  shipping_limit_date VARCHAR(255),  price REAL,  freight_value REAL)"""
queryTable(createOrd)

syntax error at or near ","
LINE 1: ...sql/data/olist_order_items_dataset.csv DELIMITER ',' CSV HEA...
                                                             ^



In [21]:
insertOrd = """COPY dim_orders FROM '/var/lib/postgresql/data/olist_order_items_dataset.csv' DELIMITER ',' CSV HEADER"""
queryTable(insertOrd)

In [23]:
queryBase("""SELECT * FROM dim_orders LIMIT 5""")

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1.0,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1.0,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1.0,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1.0,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [24]:
payment= pd.read_csv("olist_order_payments_dataset.csv")
payment.head()
#Here again the order_id will be the key. In fact would have been referenced

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [25]:
schemaGen(payment,'dim_payment')

'CREATE TABLE dim_payment (order_id VARCHAR(255),  payment_sequential NUMERIC,  payment_type VARCHAR(255),  payment_installments NUMERIC,  payment_value REAL)'

In [26]:
createPymnt = """CREATE TABLE dim_payment (order_id VARCHAR(255),  payment_sequential NUMERIC,  payment_type VARCHAR(255),  payment_installments NUMERIC,  payment_value REAL)"""
queryTable(createPymnt)

In [28]:
insertPymnt = """COPY dim_payment FROM '/var/lib/postgresql/data/olist_order_payments_dataset.csv'
                DELIMITER ',' CSV HEADER"""
queryTable(insertPymnt)

In [29]:
queryBase("""SELECT * FROM dim_payment LIMIT 5""")

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1.0,credit_card,8.0,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1.0,credit_card,1.0,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1.0,credit_card,1.0,65.71
3,ba78997921bbcdc1373bb41e913ab953,1.0,credit_card,8.0,107.78
4,42fdf880ba16b47b59251dd489d4441a,1.0,credit_card,2.0,128.45


In [30]:
reviews = pd.read_csv("olist_order_reviews_dataset.csv")
reviews.head(5)
#Will have review_id as the primary key

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [31]:
schemaGen(reviews,'dim_reviews')

'CREATE TABLE dim_reviews (review_id VARCHAR(255),  order_id VARCHAR(255),  review_score NUMERIC,  review_comment_title VARCHAR(255),  review_comment_message VARCHAR(255),  review_creation_date VARCHAR(255),  review_answer_timestamp VARCHAR(255))'

In [32]:
credimr = """CREATE TABLE dim_reviews (review_id VARCHAR(255),  order_id VARCHAR(255),  review_score NUMERIC,  review_comment_title VARCHAR(255),  review_comment_message VARCHAR(255),  review_creation_date VARCHAR(255),  review_answer_timestamp VARCHAR(255))"""
queryTable(credimr)

In [33]:
insertRev = """COPY dim_reviews FROM '/var/lib/postgresql/data/olist_order_reviews_dataset.csv'
            DELIMITER ',' CSV HEADER"""
queryTable(insertRev)

In [35]:
order_process = pd.read_csv('olist_orders_dataset.csv')
order_process.head(2)
#order_id needs to be PK

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00


In [36]:
schemaGen(order_process,'dim_process')

'CREATE TABLE dim_process (order_id VARCHAR(255),  customer_id VARCHAR(255),  order_status VARCHAR(255),  order_purchase_timestamp VARCHAR(255),  order_approved_at VARCHAR(255),  order_delivered_carrier_date VARCHAR(255),  order_delivered_customer_date VARCHAR(255),  order_estimated_delivery_date VARCHAR(255))'

In [37]:
createPRoce = """CREATE TABLE dim_process (order_id VARCHAR(255),  customer_id VARCHAR(255),  order_status VARCHAR(255),  order_purchase_timestamp VARCHAR(255),  order_approved_at VARCHAR(255),  order_delivered_carrier_date VARCHAR(255),  order_delivered_customer_date VARCHAR(255),  order_estimated_delivery_date VARCHAR(255))"""
queryTable(createPRoce)

In [38]:
insertProce = """COPY dim_process FROM '/var/lib/postgresql/data/olist_orders_dataset.csv'
                DELIMITER ',' CSV HEADER"""
queryTable(insertProce)

In [39]:
queryBase("""SELECT * FROM dim_process LIMIT 5""")

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [40]:
product = pd.read_csv('olist_products_dataset.csv')
product.head(2)
#product_ID will be the unique PK for this table

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0


In [41]:
schemaGen(product,'dim_product')

'CREATE TABLE dim_product (product_id VARCHAR(255),  product_category_name VARCHAR(255),  product_name_lenght REAL,  product_description_lenght REAL,  product_photos_qty REAL,  product_weight_g REAL,  product_length_cm REAL,  product_height_cm REAL,  product_width_cm REAL)'

In [42]:
createPdt = """CREATE TABLE dim_product (product_id VARCHAR(255),  product_category_name VARCHAR(255),  product_name_lenght REAL,  product_description_lenght REAL,  product_photos_qty REAL,  product_weight_g REAL,  product_length_cm REAL,  product_height_cm REAL,  product_width_cm REAL)"""
queryTable(createPdt)

In [44]:
insertPdt = """COPY dim_product FROM '/var/lib/postgresql/data/olist_products_dataset.csv'
                DELIMITER ',' CSV HEADER"""
queryTable(insertPdt)

In [45]:
queryBase("""SELECT * FROM dim_product LIMIT 5""")

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [46]:
sellers = pd.read_csv("olist_sellers_dataset.csv")
sellers.head(2)
#seller_id will be unique PK

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP


In [47]:
schemaGen(sellers,'dim_seller')

'CREATE TABLE dim_seller (seller_id VARCHAR(255),  seller_zip_code_prefix NUMERIC,  seller_city VARCHAR(255),  seller_state VARCHAR(255))'

In [48]:
createSel = """CREATE TABLE dim_seller (seller_id VARCHAR(255),  seller_zip_code_prefix NUMERIC,  seller_city VARCHAR(255),  seller_state VARCHAR(255))"""
queryTable(createSel)

In [49]:
insertSel = """COPY dim_seller FROM '/var/lib/postgresql/data/olist_sellers_dataset.csv'
                DELIMITER ',' CSV HEADER"""
queryTable(insertSel)

In [50]:
pdtctg = pd.read_csv("product_category_name_translation.csv")
pdtctg.head(2)

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories


In [51]:
schemaGen(pdtctg,'dim_category')

'CREATE TABLE dim_category (product_category_name VARCHAR(255),  product_category_name_english VARCHAR(255))'

In [52]:
creatCtg = """CREATE TABLE dim_category (product_category_name VARCHAR(255),  product_category_name_english VARCHAR(255))"""
queryTable(creatCtg)

In [53]:
insertCtg = """COPY dim_category from '/var/lib/postgresql/data/product_category_name_translation.csv'
            DELIMITER ',' CSV HEADER"""
queryTable(insertCtg)

In [54]:
queryBase("""SELECT * FROM dim_category LIMIT 5""")

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [55]:
geoloc = pd.read_csv("olist_geolocation_dataset.csv")
geoloc.head()
#zip_Code can be unique PK here

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [56]:
schemaGen(geoloc,'dim_geo')

'CREATE TABLE dim_geo (geolocation_zip_code_prefix NUMERIC,  geolocation_lat REAL,  geolocation_lng REAL,  geolocation_city VARCHAR(255),  geolocation_state VARCHAR(255))'

In [57]:
createGeo = """CREATE TABLE dim_geo (geolocation_zip_code_prefix NUMERIC,  geolocation_lat REAL,  geolocation_lng REAL,  geolocation_city VARCHAR(255),  geolocation_state VARCHAR(255))"""
queryTable(createGeo)

In [58]:
inserGeo = """COPY dim_geo FROM '/var/lib/postgresql/data/olist_geolocation_dataset.csv'
            DELIMITER ',' CSV HEADER"""
queryTable(inserGeo)

In [59]:
queryBase("""SELECT * FROM dim_geo LIMIT 5""")

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037.0,-23.545622,-46.639294,sao paulo,SP
1,1046.0,-23.546082,-46.644820,sao paulo,SP
2,1046.0,-23.546130,-46.642952,sao paulo,SP
3,1041.0,-23.544392,-46.639500,sao paulo,SP
4,1035.0,-23.541578,-46.641605,sao paulo,SP


### A rare dataset or is the helper function doing better

The above 9 datasets were sent into the database in matter 30 minutes. The data was already segregated based on the various processes, and PKs were identifiable. The purpose of the tables were clear. Over all the database design can easily follow the dataset columns to be efficient.